In [1]:
import numpy as np
import sklearn
import torch
import pandas as pd

In [2]:
# Load data

data = pd.read_csv('data/raw/events-20k.csv')

data.head()

,q1,x1,y1,z1,px1,py1,pz1,x2,y2,z2,px2,py2,pz2,q2,vtx,vty,vtz,vpx,vpy,vpz
0,1,14.03680,9.14611,594.467,2.227900,0.682157,40.16110,96.92970,31.9730,1936.99,2.612100,0.682219,40.11380,1,-0.094969,-0.173778,37.7394,-0.718459,0.855686,49.2060
1,1,-15.12700,-25.56580,594.718,-0.423330,-2.644100,55.02350,-19.27280,-88.2689,1899.93,-0.035456,-2.641810,55.00070,1,-0.873625,0.590022,33.2498,-3.135350,-3.081690,62.5317
2,1,-4.48184,-5.02827,594.616,0.569838,-0.706143,72.74330,10.18230,-17.6988,1899.86,0.956272,-0.705887,72.71820,1,0.363496,0.541102,27.1000,-2.388700,-0.748944,80.4553
3,1,-45.93980,18.25010,594.912,-0.013079,0.509187,9.29352,-11.46980,91.4206,1936.73,0.374477,0.506969,9.30575,1,-0.047285,0.010607,56.4399,-3.557140,0.214992,18.2017
4,1,-19.95970,28.69740,594.706,0.195606,1.114430,22.93230,6.27534,93.8775,1936.75,0.583209,1.111620,22.90280,1,-0.263670,1.339230,41.8576,-2.991010,1.618660,30.3003


In [3]:
X, y = data[['q1', 'x1', 'y1', 'z1', 'px1', 'py1', 'pz1', 'x2', 'y2', 'z2', 'px2', 'py2', 'pz2']].to_numpy(), data[['vtz']].to_numpy()

In [4]:
X.shape, y.shape

((25380, 13), (25380, 1))

In [5]:
# Train test split

from sklearn.model_selection import train_test_split


X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25)

print('training:')
print(X_train.shape)
print(y_train.shape)

print('validation:')
print(X_val.shape)
print(y_val.shape)

print('test:')
print(X_test.shape)
print(y_test.shape)

training:
(15228, 13)
(15228, 1)
validation:
(5076, 13)
(5076, 1)
test:
(5076, 13)
(5076, 1)


In [6]:
# We scale the input

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [7]:
# Finally, let's construct the model

import torch.nn.functional as F
from torch.nn import Linear

class vertexFinder(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(vertexFinder, self).__init__()
        # Let's define our layers here.
        self.fc1 = torch.nn.Linear(in_features, 16, bias=True) # Define the first layer (input: in_features and 32 outputs)
        self.fc2 = torch.nn.Linear(16, 3, bias=True) # Define the second hidden layer 
        self.fc3 = torch.nn.Linear(3, 3, bias=True) # Define the third hidden layer 
        self.fc4 = torch.nn.Linear(3, out_features, bias=True) # Define the output layer 

    def forward(self, x):
        # Let's define out forward pass
        x = F.relu(self.fc1(x)) # Pass through the first layer and relu activation
        x = F.relu(self.fc2(x)) # Now through the second
        x = F.relu(self.fc3(x)) # And third
        x = F.relu(self.fc4(x)) # Our output should be linear
        return x

In [8]:
net = vertexFinder(13, 1)

In [9]:
print(net)

vertexFinder(
  (fc1): Linear(in_features=13, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=1, bias=True)
)


In [10]:
print(net.fc3.weight.shape)

torch.Size([3, 3])


In [11]:
# model_scripted = torch.jit.script(net) 
# model_scripted.save('myVertexFinder.pt')

In [12]:
# # Let's visualize the computational graph

# from torchviz import make_dot

# net.eval()
# yhat = net(torch.zeros(1, 13))
# make_dot(yhat, params=dict(list(net.named_parameters())))

In [13]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64

train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':1.5f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{avg' + self.fmt + '} ({name})'
        return fmtstr.format(**self.__dict__)

In [15]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
epochs = 10

In [16]:
acc, loss = AverageMeter('Accuracy'), AverageMeter('Loss')
train_loss, val_acc, val_loss = [], [], []

# Iterate over the dataset <epochs> times
for epoch in range(epochs):

    # Set the model to training mode
    net.train()
    # Reset our meters
    loss.reset()
    acc.reset()

    # Iterate over batches
    for inputs, targets in train_dataloader:

        # Remove previous gradients
        optimizer.zero_grad()
        
        # Feed forward the input
        outputs = net(inputs)
        

        # Compute the loss and accuracy
        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)
        
        accuracy = (targets == outputs).sum()
        acc.update(accuracy.data)
        

        # Compute the gradients
        loss_batch.backward()

        # Update parameters
        optimizer.step()
        
    train_loss.append(loss.avg)
    
    print("Training : Epoch , Loss, Acc : {}, {}, {}".format(epoch+1, loss.avg, acc.avg))
    
    # Validation for each epoch
    net.eval()
    loss.reset()
    acc.reset()

    for inputs, targets in val_dataloader:

        outputs = net(inputs)

        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)

        preds = torch.argmax(outputs, dim=-1)
        accuracy = (torch.argmax(targets, dim=-1) == preds).sum() / len(targets)
        acc.update(accuracy.data)
        

    val_loss.append(loss.avg)
    val_acc.append(acc.avg)
    
    print("Validate : Epoch , Loss, Acc : {}, {}, {}".format(epoch+1, loss.avg, acc.avg))

RuntimeError: The size of tensor a (64) must match the size of tensor b (60) at non-singleton dimension 0

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use(hep.style.ROOT)

# Let's draw loss and accuracy for the training and validation

def draw_loss(data_train, data_val, data_acc, label="Loss"):
    """Plots the training and validation loss"""

    fig, ax1 = plt.subplots(figsize=(5, 5))
    ax1.set_xlabel("Epoch", horizontalalignment='right', x=1.0)
    ax1.set_ylabel("Loss", horizontalalignment='right', y=1.0)
    ax1.set_yscale('log')
    ax1.tick_params(axis='y', labelcolor='red')
    ax1.plot(data_train,
             color='red',
             label='Training loss')
    ax1.plot(data_val,
             color='blue',
             label='Validation loss')
    ax2 = ax1.twinx()
    ax2.set_ylabel('Accuracy', color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    ax2.plot(data_acc,
             color='green',
             label='Accuracy')
    ax1.legend(loc='lower left')
    ax2.legend(loc='upper left')
    plt.show()

draw_loss(train_loss, val_loss, val_acc)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = net(torch.tensor(X_test).unsqueeze(0).float())

print("Accuracy for the test set: {0:.2f}".format(
    accuracy_score(
        np.argmax(y_test, axis=1),
        torch.argmax(y_pred, dim=-1).squeeze().numpy())
))

In [ ]:
plt.hist(y_pred.detach().numpy().squeeze())
plt.show()